In [12]:
import pandas as pd

df = pd.read_parquet('./train-00000-of-00001.parquet',engine='pyarrow')

print(df.loc[0, 'prompt'])
print(df.loc[0, 'context'])
print(df.loc[0, 'answer'])

df.info()


What is the total volume of timber sold by each salesperson, sorted by salesperson?
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');
SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126641 entries, 0 to 126640
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   prompt   126641 non-null  object
 1   context  126641 non-null  object
 2   an

In [13]:
import json
import ijson

question_set = {}
instruction_set = set()
with open('./dev_sql.json', 'rt') as f:
	for item in ijson.items(f, 'item'):
		question_set[item['input'].strip()] = item['output'].strip()
		instruction_set.add(item['instruction'].strip())


output = []
system_prompt = "Given the database schema and the user question, generate the corresponding SQL query."
sql_prompt_template = """\[SCHEMA]
{schema}
\[QUESTION]
{question}
"""

cnt = 0
for item in df.iterrows():
	if item[1]['prompt'].strip() in question_set:
		cnt += 1
		continue
	output.append({
		'conversations': [
			{
				'from': 'user',
				'value': sql_prompt_template.format(schema=item[1]['context'], question=item[1]['prompt'])
			},
			{
				'from': 'assistant',
				'value': item[1]['answer']
			}
		],
		'system': system_prompt
	})

with open('../text2sql.json', 'w') as f:
	json.dump(output, f, indent=2)

print(f'{len(output)} examples are added to train set.')
print(f'{cnt} questions are already in the val dataset.')


126285 examples are added to train set.
356 questions are already in the val dataset.


In [14]:
print(len(question_set))
print(len(instruction_set))


1034
145
